In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.model_selection import KFold
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
import os
import pandas as pd
import numpy as np
import joblib

In [0]:
base = "drive/My Drive/외부/공모전/[기상] AI프렌즈 시즌1 온도 추정 경진대회/"

train = pd.read_csv(base + "train.csv")
X_cols = [col for col in train.columns if 'X' in col]

In [0]:
# display(train.head(5))

In [0]:
# train.corr()

In [0]:
train_data = pd.DataFrame(columns=X_cols + ['Y'])
val_data = pd.DataFrame(columns=X_cols + ['Y'])

start = 0; end = 4320

for i in range(19):
  Y_col = "Y{0:02}".format(i)

  if i == 18:
    start = 4320; end = 4752

  temp_table = train[[*X_cols, Y_col]][start:end]
  temp_table.rename(columns = {Y_col : 'Y'}, inplace = True)
  
  if i == 18:
    val_data = pd.concat([val_data, temp_table], axis=0)
  else:
    train_data = pd.concat([train_data, temp_table], axis=0)

In [0]:
print(train_data.isnull().any().sum())
print(val_data.isnull().any().sum())

0
0


In [0]:
# Y_data_format = set()

# for data in train_data:
#   for it in data.Y.unique():
#     Y_data_format.add(it)

# print(sorted(Y_data_format))

[3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 53.5, 54.0, 54.5]


In [0]:
# for idx, data in enumerate(train_data):
#   data['Y'] = data['Y'].apply(lambda x: x * 2)
#   train_data[idx] = data.astype({'Y': 'int'})

In [0]:
def train_model(clf):
  clf.fit(train_data[X_cols], train_data['Y'])

def val_model(clf):
  clf.fit(val_data[X_cols], val_data['Y'])

def get_test_accuracy(model):
  pred = model.predict(val_data[X_cols])
  mae = mean_absolute_error(val_data['Y'], pred)

  return mae

def set_model(Model, *args):
  clf = Model(args)
  train_model(clfclf)
  print(get_test_accuracy(reg))

def set_submit(clf, name):
  val_model(clf)
  get_submission(clf, name)

def get_submission(model, name="None"):
  submit = pd.read_csv(base + 'sample_submission.csv')
  test = pd.read_csv(base + "test.csv")
  submit['Y18'] = model.predict(test[X_cols])

  submit.to_csv(base + name + ".csv", index=None)

In [0]:
lgb_train = lgb.Dataset(train_data[X_cols], train_data['Y'])
lgb_all_train = lgb.Dataset(train_data[X_cols] + val_data[X_cols], train_data['Y'] + val_data['Y'])

lgb_param = {
    "objective" : "regression",
    "metrics" : 'mae',
    'learning_rae': 0.01
}

In [0]:
cv_result = lgb.cv(
    lgb_param,
    lgb_train,
    num_boost_round=99999,
    nfold = 5,
    early_stopping_rounds=100,
    stratified=False,
    verbose_eval=10
)

[10]	cv_agg's l1: 3.53484 + 0.0187347
[20]	cv_agg's l1: 2.89619 + 0.0185454
[30]	cv_agg's l1: 2.74108 + 0.0202925
[40]	cv_agg's l1: 2.70014 + 0.0213839
[50]	cv_agg's l1: 2.68919 + 0.0218647
[60]	cv_agg's l1: 2.68731 + 0.021696
[70]	cv_agg's l1: 2.68884 + 0.0212758
[80]	cv_agg's l1: 2.69126 + 0.0211839
[90]	cv_agg's l1: 2.69373 + 0.0208536
[100]	cv_agg's l1: 2.69657 + 0.020583
[110]	cv_agg's l1: 2.69937 + 0.0206043
[120]	cv_agg's l1: 2.70201 + 0.0207849
[130]	cv_agg's l1: 2.70481 + 0.0206771
[140]	cv_agg's l1: 2.70754 + 0.0203533
[150]	cv_agg's l1: 2.71021 + 0.020403


In [0]:
lgb_model = lgb.train(
    lgb_param,
    lgb_train,
    num_boost_round=len(cv_result['l1-mean'])
)

In [0]:
print(get_test_accuracy(lgb_model))
lgb_model = lgb.train(
    lgb_param,
    lgb_all_train,
    num_boost_round=len(cv_result['l1-mean'])
)

1.9549113815263222


In [0]:
get_submission(lgb_model, "lightgbm")

In [0]:
from sklearn.linear_model import TheilSenRegressor

clf = TheilSenRegressor()
train_model(clf)
print(get_test_accuracy(clf))

1.9489954819486195


In [0]:
val_model(clf)
get_submission(clf, "TheilSenRegressor")

In [0]:
from sklearn.linear_model import Lasso

clf = Lasso(alpha=1.0)
train_model(clf)
print(get_test_accuracy(clf))

2.0049235971358708


In [0]:
val_model(clf)
get_submission(clf, "Lasso")

In [0]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
train_model(clf)
print(get_test_accuracy(clf))

Ridge(alpha=100.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [0]:
val_model(clf)
get_submission(clf, "ridge")

1.9293391844691807


## Y18 validation

### TheilSenRegressor
- 1.9489954819486195

### Lasso
- alpha 0.1 : 1.989884090429618

### Ridge
- alpha 1.0 : 1.9050152629571775

## test Data Score


### TheilSenRegressor
- 803.0230802576

### Lasso
- 118.4287797422

### Ridge
- 8.1227563099